In [ ]:
import numpy as np
from skimage.io import imread, imsave
from skimage.util import img_as_ubyte
from skimage import img_as_float
import os
import csv

In [ ]:
def svd_compress(image, k):
    im_f = img_as_float(image)
    U, s, Vt = np.linalg.svd(im_f, full_matrices=False)
    U_reduced = U[:, :k]
    s_reduced = s[:k]
    Vt_reduced = Vt[:k, :]
    
    return U_reduced, s_reduced, Vt_reduced

def svd_decompress(U, s, Vt):
    im_reconstructed = np.dot(U, np.dot(np.diag(s), Vt))
    im_reconstructed_normalized = (im_reconstructed - im_reconstructed.min()) / (im_reconstructed.max() - im_reconstructed.min())
    im_reconstructed_ubyte = img_as_ubyte(im_reconstructed_normalized)
    return im_reconstructed_ubyte

def format_size(size_in_bytes):
    for unit in ['B', 'KB', 'MB', 'GB', 'TB']:
        if size_in_bytes < 1024.0:
            break
        size_in_bytes /= 1024.0
    return f"{size_in_bytes:.2f} {unit}"

def mean_squared_error(original_image, decompressed_image):
    if original_image.shape != decompressed_image.shape:
        raise ValueError("Both images must have the same shape for MSE calculation.")
    mse = np.mean((original_image - decompressed_image) ** 2)
    return mse

def csv_write(file_path, adjusted_list, mean_score_list):
    csv_file_path = file_path  
    with open(csv_file_path, 'w', newline='') as csv_file:
        csv_writer = csv.writer(csv_file)
        for k, size_dct in enumerate(adjusted_list):
            csv_writer.writerow([k+5, size_dct, mean_score_list[k]])

In [ ]:
image_path = ''

In [ ]:
mean_score_svd = list()
compress_size_svd = list()
image = imread(image_path)
original_size = format_size(os.path.getsize(image_path))
for k_value in range(5, 205):
    U_reduced, s_reduced, Vt_reduced = svd_compress(image, k=k_value)
    image_reconstructed = svd_decompress(U_reduced, s_reduced, Vt_reduced)
    imsave(f'svd_compressed_{k_value}.jpg', image_reconstructed)
    compressed_size = format_size(os.path.getsize(f'svd_compressed_{k_value}.jpg'))
    mse_score = mean_squared_error(img_as_float(image), img_as_float(image_reconstructed))
    print(f"K-Value—{k_value} || Compressed Size-{compressed_size} || Mean Score—{mse_score}")
    mean_score_svd.append(mse_score)
    compress_size_svd.append(compressed_size)

In [ ]:
adjusted_svd = [float(value.split(' ')[0]) for value in compress_size_svd]
csv_write('dct.ipynb', adjusted_svd, mean_score_svd)